In [ ]:
import os
import shutil
import time

os.environ["LOGURU_LEVEL"] = "ERROR"

## Modify Resources(gpus) in EmbeddingGeneration to speedup

Please note this is dependent on model used and GPU sku. 

Assuming < 50% is being utilized we can expect speedup 

In [2]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"

In [3]:
from nemo_curator.stages.resources import Resources
from nemo_curator.stages.text.embedders import EmbeddingCreatorStage
from nemo_curator.stages.text.models.utils import format_name_with_suffix

embedding_creator = EmbeddingCreatorStage(model_identifier=model_name, model_inference_batch_size=256)

embedding_creator_half_gpu = EmbeddingCreatorStage(model_identifier=model_name, model_inference_batch_size=256).with_(
    {format_name_with_suffix(model_name, "_model"): {"resources": Resources(gpus=0.5)}}
)

In [ ]:
import uuid

from nemo_curator.backends.experimental.ray_data import RayDataExecutor
from nemo_curator.core.client import RayClient
from nemo_curator.pipeline import Pipeline
from nemo_curator.stages.text.io.reader.parquet import ParquetReader
from nemo_curator.stages.text.io.writer.parquet import ParquetWriter

input_path = "./tinystories_train_parquet"
output_path = "./output_path/"
ray_temp_dir = "/tmp"  # noqa: S108

shutil.rmtree(output_path, ignore_errors=True)

time_taken = {}
for _embedding_creator, _name in [(embedding_creator, "full_gpu"), (embedding_creator_half_gpu, "half_gpu")]:
    # Ensure RAY_ADDRESS is cleared before starting a new cluster
    os.environ["RAY_ADDRESS"] = ""
    ray_temp_dir_path = os.path.join(ray_temp_dir, str(uuid.uuid4())[:6])
    pipeline = Pipeline(
        name=_name,
        stages=[
            ParquetReader(
                file_paths=input_path,
                fields=["text"],
            ),
            _embedding_creator,
            ParquetWriter(os.path.join(output_path, f"{_name}_tinystories")),
        ],
    )
    with RayClient(
        num_cpus=64,
        num_gpus=4,
        ray_temp_dir=ray_temp_dir_path,
        ray_dashboard_host="0.0.0.0",  # noqa: S104
    ) as client:
        t0 = time.time()
        pipeline.run(RayDataExecutor())
        t1 = time.time()
        time_taken[_name] = t1 - t0
        print(f"Time taken for {_name}: {t1 - t0} seconds")
    # Small delay to ensure Ray cluster is fully cleaned up before next iteration
    time.sleep(2)

2025-11-20 23:44:39,539	INFO worker.py:1691 -- Using address 127.0.1.1:6380 set in the environment variable RAY_ADDRESS
2025-11-20 23:44:39,548	INFO worker.py:1832 -- Connecting to existing Ray cluster at address: 127.0.1.1:6380...


2025-11-20 23:44:40,950	INFO usage_lib.py:447 -- Usage stats collection is disabled.
2025-11-20 23:44:40,950	INFO scripts.py:914 -- Local node IP: 127.0.1.1


[2025-11-20 23:44:45,799 W 216844 216844] global_state_accessor.cc:505: Some processes that the driver needs to connect to have not registered with GCS, so retrying. Have you run 'ray start' on this node?
[2025-11-20 23:44:46,801 W 216844 216844] global_state_accessor.cc:505: Some processes that the driver needs to connect to have not registered with GCS, so retrying. Have you run 'ray start' on this node?


2025-11-20 23:44:47,452	SUCC scripts.py:950 -- --------------------
2025-11-20 23:44:47,452	SUCC scripts.py:951 -- Ray runtime started.
2025-11-20 23:44:47,452	SUCC scripts.py:952 -- --------------------
2025-11-20 23:44:47,452	INFO scripts.py:954 -- Next steps
2025-11-20 23:44:47,452	INFO scripts.py:957 -- To add another node to this Ray cluster, run
2025-11-20 23:44:47,452	INFO scripts.py:960 --   ray start --address='127.0.1.1:6380'
2025-11-20 23:44:47,452	INFO scripts.py:969 -- To connect to this Ray cluster:
2025-11-20 23:44:47,452	INFO scripts.py:971 -- import ray
2025-11-20 23:44:47,453	INFO scripts.py:972 -- ray.init(_node_ip_address='127.0.1.1')
2025-11-20 23:44:47,453	INFO scripts.py:984 -- To submit a Ray job using the Ray Jobs CLI:
2025-11-20 23:44:47,453	INFO scripts.py:985 --   RAY_API_SERVER_ADDRESS='http://127.0.1.1:8266' ray job submit --working-dir . -- python my_script.py
2025-11-20 23:44:47,453	INFO scripts.py:994 -- See https://docs.ray.io/en/latest/cluster/running

[2025-11-20 23:44:47,802 I 216844 216844] global_state_accessor.cc:487: This node has an IP address of 127.0.1.1, but we cannot find a local Raylet with the same address. This can happen when you connect to the Ray cluster with a different IP address or when connecting to a container.
2025-11-20 23:44:47,805	INFO worker.py:2012 -- Connected to Ray cluster.
/raid/praateekm/NeMo-Curator/.venv/lib/python3.12/site-packages/ray/_private/worker.py:2051: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 9398.65it/s]
2025-11-20 23:44:59,863	WARNING util.py:597 -- The argument ``concurrency`` is deprecated in Ray 2.51. Please specify argument ``compute`` instead. For more information, see https://docs.ray.io/en/master/data/tr

...

- MapBatches(ParquetReaderStageTask)->MapBatches(TokenizerStageActor) 3: 0.00 row [00:00, ? row/s]

- MapBatches(EmbeddingModelStageActor) 4: 0.00 row [00:00, ? row/s]

- MapBatches(ParquetWriterTask) 5: 0.00 row [00:00, ? row/s]

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 11785.06it/s]zerStageActor)) pid=258816) 
2025-11-20 23:50:48,058	INFO streaming_executor.py:298 -- ✔️  Dataset dataset_6_0 execution finished in 133.19 seconds
2025-11-20 23:50:48,064	INFO util.py:257 -- Exiting prefetcher's background thread


Time taken for half_gpu: 153.33278346061707 seconds


In [5]:
for run_name, run_time in time_taken.items():
    print(f"Time taken for {run_name}: {run_time:.2f} seconds")

Time taken for full_gpu: 213.12 seconds
Time taken for half_gpu: 153.33 seconds
